# Homework 3

## Basic

In [270]:
import numpy as np

In [293]:
def cholesky(A):
    n = len(A)

    # Create zero matrix for L
    L = [[0.0] * n for i in range(n)]

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                # LaTeX: l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                # LaTeX: l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right)
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

In [271]:
def value_hw3(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    if n == 2:
        A = np.array([[sigma[0], sigma[1] * rho[0][1]], [0, sigma[1] * np.sqrt(1 - rho[0][1] ** 2)]])
    else:
        sigma = np.reshape(sigma, (len(sigma), 1))
        cov_mat = sigma * sigma.T * np.array(rho)
        A = np.linalg.cholesky(cov_mat).T
    V_n = []
    for j in range(repeat):
        V_list = []
        for i in range(num):
            z_T = np.matmul(np.random.randn(1, n) ,A) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - np.array(q) - 0.5 * np.array(sigma) ** 2) * T + z_T)
            V_list.append((np.amax(S_list) - K) * (1 if np.amax(S_list) > K else 0))
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()]

In [455]:
value_hw3(100, 0.1, 0.5, 10000, 20, 2, [95]*2, [0.05]*2, [0.5]*2, [[1, 1], [1, 1]])

(11.979472840414452,
 0.18063107279594837,
 [11.618210694822555, 12.34073498600635])

In [456]:
value_hw3(100, 0.1, 0.5, 10000, 20, 2, [95]*2, [0.05]*2, [0.5]*2, [[1, -1], [-1, 1]])

(23.88460047313434,
 0.23208144260205285,
 [23.420437587930234, 24.34876335833845])

In [457]:
value_hw3(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.4779925069505,
 0.3331820397781881,
 [29.811628427394123, 31.144356586506877])

## Bonus 1

In [332]:
def value_hw3_bonus(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    if n == 2:
        A = np.array([[sigma[0], sigma[1] * rho[0][1]], [0, sigma[1] * np.sqrt(1 - rho[0][1] ** 2)]])
    else:
        sigma = np.reshape(sigma, (len(sigma), 1))
        cov_mat = sigma * sigma.T * np.array(rho)
        A = np.linalg.cholesky(cov_mat).T
    V_n = []
    for j in range(repeat):
        V_list = []
        rand_list = np.random.randn(int(num/2), n)
        rand_list = np.append(rand_list, -rand_list)
        rand_list = rand_list / rand_list.std()
        for i in range(num):
            z_T = np.matmul(rand_list[i * n : (i+1) * n], A) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - np.array(q) - 0.5 * np.array(sigma) ** 2) * T + z_T)
            V_list.append(payoff(S_list, K))
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()]

In [458]:
value_hw3_bonus(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.402896265234755,
 0.10722120988560428,
 [30.188453845463545, 30.617338685005965])

## Bonus 2

In [512]:
def value_hw3_bonus2(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    if n == 2:
        A = np.array([[sigma[0], sigma[1] * rho[0][1]], [0, sigma[1] * np.sqrt(1 - rho[0][1] ** 2)]])
    else:
        sigma = np.reshape(sigma, (len(sigma), 1))
        cov_mat = sigma * sigma.T * np.array(rho)
        A_ = np.linalg.cholesky(cov_mat).T
    V_n = []
    for j in range(repeat):
        V_list = []
        rand_list = np.random.randn(num, n)
        rand_ = rand_list - (np.sum(rand_list, axis=0) / num)
        rand_cov = np.cov(rand_.T)
        A = np.array(cholesky(rand_cov)).T
        A_inv = np.linalg.inv(A)
        rand__ = np.matmul(A_inv, rand_.T)
        for i in range(num):
            z_T = np.matmul(rand__[:, i], A_) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - q[0] - 0.5 * (sigma[0] ** 2)) * T + z_T)
            V_list.append(payoff(S_list, K))
#             print(z_T, S_list)
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()], V_n

In [515]:
value_hw3_bonus2(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.387713134402752,
 0.1217987678389269,
 [30.1441155987249, 30.631310670080605],
 array([30.27939832, 30.30619213, 30.53925249, 30.40875576, 30.3509485 ,
        30.58899158, 30.32264754, 30.37520656, 30.26324764, 30.29746406,
        30.27958896, 30.276415  , 30.66564758, 30.24521829, 30.48911105,
        30.25423222, 30.55329229, 30.3781506 , 30.47390666, 30.40659548]))